In [14]:
import torch
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.optim as optim

# 数据
batch_size = 64

# 将图片转换为张量
transform = transforms.Compose([
    # 将PIL图像换为PyTorch张量，它会将像素值从 [0, 255] 范围缩放到 [0.0, 1.0] 范围
    transforms.ToTensor(),
    # 标准化 (x-mean)/std   mean    std      让数据更符合正态分布，加速训练。
    transforms.Normalize((0.1307,),(0.3081,))
])

train_dataset = datasets.MNIST(root='D:/code/datasets/minist/',
                               train=True,
                               download=True,
                               transform=transform)
train_loader = DataLoader(train_dataset,
                          shuffle=True,
                          batch_size=batch_size)
test_dataset = datasets.MNIST(root='D:/code/datasets/minist/',
                               train=False,
                               download=True,
                               transform=transform)
test_loader = DataLoader(test_dataset,
                          shuffle=False,
                          batch_size=batch_size)


![jupyter](./images/minist_Modelframe.png)

In [15]:
# 模型
class Net(torch.nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.l1 = torch.nn.Linear(784,512)
        self.l2 = torch.nn.Linear(512,256)
        self.l3 = torch.nn.Linear(256,128)
        self.l4 = torch.nn.Linear(128,64)
        self.l5 = torch.nn.Linear(64,10)
    
    def forward(self,x):
        # 784 == 28*28 为图片像素点数  -1:自动得出行数
        x = x.view(-1,784)
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = F.relu(self.l3(x))
        x = F.relu(self.l4(x))
        # outputs不需要激活
        return self.l5(x)
        
model =Net()

In [16]:
# 损失函数和优化器
criterion = torch.nn.CrossEntropyLoss()
# momentum：动量参数，通常用于加速收敛。通过保留上一次的更新方向，可以在一定程度上克服梯度下降中的震荡问题。动量越大，更新时方向越平滑。
optimizer = optim.SGD(model.parameters(),lr=0.01,momentum=0.5)

In [ ]:
# 训练
def train(epoch,loss_list):
    running_loss = 0
    for batch_idx,(inputs,target) in enumerate(train_loader,0):
        # 由于将单次训练打包为函数,无法确定是否为首轮训练，故提前将梯度归零
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs,target)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if batch_idx %300 ==299:
            print('[%d, %5d]    loss:%.3f' %(epoch+1,batch_idx+1,running_loss/300))
            running_loss =0


In [18]:
# 测试
def test():
    correct = 0
    total = 0
    # 禁止求梯度，防止额外生成计算图
    with torch.no_grad():
        for (images,labels) in test_loader:
            outputs = model(images)
            # torch.max(i,dim=j)取出张量i中在j维度中的所有最大值,0维度为列1维度为行,此处为取出每行的最大值即可能性最大的那个数字
            _, predicted = torch.max(outputs.data,dim=1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            print('Accuracy on test set : %d %%' %(100*correct/total))

In [ ]:
if __name__ =='__main__':
    for epoch in range(10):
        train(epoch)
        if epoch % 10 == 9:
            test()

TypeError: train() takes 1 positional argument but 2 were given

NameError: name 'loss_list' is not defined

<Figure size 1000x600 with 0 Axes>